In [69]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from openbabel import openbabel
import pubchempy as pcp
import numpy as np
import py3Dmol
import os
import re
import json
from io import StringIO
from Bio.PDB import PDBParser, PDBList
import ipywidgets as widgets
from ipywidgets import interact, fixed, IntSlider, Text, Dropdown, ToggleButton, Button, FloatSlider, Checkbox, SelectMultiple
from IPython.display import display
import streamlit as st
obMol = openbabel.OBMol()
obConv = openbabel.OBConversion()
"""IMPORTANT: DO NOT USE ANY OTHER VARIABLES NAMED obMol OR obConv!!!"""

from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdCIPLabeler
from rdkit.Chem import rdAbbreviations
IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.ipython_useSVG=True
IPythonConsole.molSize = 300,300



import pymsym

The `DataCache` class is a class that stores all the identifiers that were previously used or converted. It helps keep a tab on what molecules were used and their identifiers, as well as help reduce computation time for the other functions in this package. with `cache.print_data()` (or whatever name is used to initialize the data cache), select which data type you want to print (`smiles`, `cid`, `inchi`, `inchikey`, `name` or `all` if you want to see all data types).

In [70]:
class DataCache:
    VALID_KEYS = ['smiles', 'cid', 'inchi', 'inchikey', 'name']

    def __init__(self):
        self.cache = {}

    def add_data(self, key, value):
        if not isinstance(key, str):
            raise ValueError("Key must be a string")
        if key.lower() not in self.VALID_KEYS:
            raise ValueError(f"Invalid key '{key}' specified")
        key = key.lower()
        if key not in self.cache:
            self.cache[key] = [value]
        else:
            if value not in self.cache[key]:
                self.cache[key].append(value)

    def get_data(self, key):
        if not isinstance(key, str):
            raise ValueError("Key must be a string")
        key = key.lower()
        if key not in self.cache:
            raise KeyError(f"Key '{key}' not found in cache")
        return self.cache.get(key)

    def print_data(self, target):
        if target not in self.VALID_KEYS + ['all']:
            raise ValueError(f"Invalid target '{target}' specified")
        
        try:
            if target == 'smiles':
                print(f"Data type: SMILES; Data: {self.cache['smiles']}")
            elif target == 'cid':
                print(f"Data type: CID; Data: {self.cache['cid']}")
            elif target == 'inchi':
                print(f"Data type: InChI; Data: {self.cache['inchi']}")
            elif target == 'inchikey':
                print(f"Data type: InChIKey; Data: {self.cache['inchikey']}")
            elif target == 'name':
                print(f"Data type: Name; Data: {self.cache['name']}")
            elif target == 'all':
                for key, value in self.cache.items():
                    print(f"Data type: {key}; Data: {value}")
        except KeyError as e:
            print(f"Error: {e}")

The `converter` class helps convert from one identifier to another. It also is able to get sdf files from pubchem for 3D visualization and is able to convert to an xyz and z-matrix format for computational chemistry purposes. <br>

This class will be used later on in the `identify_chemical_identifier(input_string)` function to convert any identifier type to the name of the molecule

In [71]:
cache = DataCache()
sdf_cache = {}
protein_cache = {}

In [72]:
class converter:
    def __init__(self, data: str, data_type: str, data_cache: DataCache):
        """Initializes the converter with the input data and its type"""
        VALID_DATA_TYPES = ['name', 'smiles', 'inchi', 'inchikey', 'cid', 'xyz', 'zmat', 'sdf']
        self.data = data
        self.data_type = data_type.lower()
        self.data_cache = data_cache
        if self.data_type not in VALID_DATA_TYPES:
            raise ValueError(f"Invalid data type '{self.data_type}' specified")

    def convert(self, target_format: str):
        """Converts the input data to the target format"""
        VALID_DATA_TYPES = ['name', 'smiles', 'inchi', 'inchikey', 'cid', 'xyz', 'zmat', 'sdf']
        target_format = target_format.lower()
        if target_format not in VALID_DATA_TYPES:
            raise ValueError(f"Invalid target format '{target_format}' specified")
            
        if self.data_type == 'name':
            self.data_cache.add_data('Name', self.data)
            smiles = self.name_to_smiles()
            sdf = self.name_to_sdf()
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.data
                
        elif self.data_type == 'smiles':
            self.data_cache.add_data('SMILES', self.data)
            smiles = self.data
            sdf = self.smiles_to_sdf()
            if target_format == 'smiles':
                return self.data
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name1()
                
        elif self.data_type == 'inchi':
            self.data_cache.add_data('InChi', self.data)
            smiles = self.inchi_to_smiles()
            sdf = self.inchi_to_sdf()
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.data
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name2(smiles)
                
        elif self.data_type == 'inchikey':
            self.data_cache.add_data('InChiKey', self.data)
            smiles = self.inchikey_to_smiles()
            sdf = self.inchikey_to_sdf()
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.data
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name2(smiles)
                
        elif self.data_type == 'cid':
            self.data_cache.add_data('CID', self.data)
            smiles = self.cid_to_smiles()
            sdf = self.cid_to_sdf()
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.data
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name2(smiles)

    def name_to_smiles(self):
        """Converts molecule name to SMILES"""
        try:
            c = pcp.get_compounds(self.data, 'name')
            smiles = c[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def cid_to_smiles(self):
        """Converts CID to SMILES"""
        try:
            cid = pcp.get_compounds(self.data, 'cid')
            smiles = cid[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def smiles_to_name1(self):
        """Converts SMILES to molecule name"""
        try:
            smi = pcp.get_compounds(self.data, 'smiles')
            name = smi[0].iupac_name
            self.data_cache.add_data('Name', name)
            return name
        except IndexError:
            return None

    def smiles_to_name2(self, smiles):
        """Converts SMILES to molecule name"""
        try:
            smi = pcp.get_compounds(smiles, 'smiles')
            name = smi[0].iupac_name
            self.data_cache.add_data('Name', name)
            return name
        except IndexError:
            return None

    def inchi_to_smiles(self):
        """Converts InChi to SMILES"""
        try:
            ic = pcp.get_compounds(self.data, 'inchi')
            smiles = ic[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def inchikey_to_smiles(self):
        """Converts InChiKey to SMILES"""
        try:
            ick = pcp.get_compounds(self.data, 'inchikey')
            smiles = ick[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def smiles_to_cid(self, smiles):
        """Converts SMILES to CID"""
        try:
            c = pcp.get_cids(smiles, 'smiles', list_return='flat')
            cid = c[0]
            self.data_cache.add_data('CID', cid)
            return cid
        except IndexError:
            return None
        
    def smiles_to_sdf(self):
        """Converts SMILES to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'smiles', overwrite=True, record_type='3d')
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                pcp.download('SDF', file_path, self.data, 'smiles', overwrite=True)
                with open(file_path, 'r') as f:
                    return f.read()
        except IndexError:
            return None

    def name_to_sdf(self):
        """Converts name to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'name', overwrite=True, record_type='3d')
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def inchi_to_sdf(self):
        """Converts inchi to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'inchi', overwrite=True, record_type='3d')
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def inchikey_to_sdf(self):
        """Converts inchikey to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'inchikey', overwrite=True, record_type='3d')
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def cid_to_sdf(self):
        """Converts name to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'cid', overwrite=True, record_type='3d')
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def smiles_to_inchi(self, smiles):
        """Converts SMILES to InChI"""
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("smiles", "inchi")
            obConv.ReadString(obMol, smiles)
            ic = obConv.WriteString(obMol)
            self.data_cache.add_data('InChi', ic)
            return ic
        except IndexError:
            return None

    def smiles_to_inchikey(self, smiles):
        """Converts SMILES to InChiKey"""
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("smiles", "inchikey")
            obConv.ReadString(obMol, smiles)
            ick = obConv.WriteString(obMol)
            self.data_cache.add_data('InChiKey', ick)
            return ick
        except IndexError:
            return None

    def sdf_to_xyz(self, sdf, smiles):
        """Converts SDF to XYZ"""
        name = self.smiles_to_name2(smiles)
        directory = './3Dfiles/'
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("sdf", "xyz")
            obConv.ReadString(obMol, sdf)
            xyz = obConv.WriteString(obMol)
            if not os.path.exists(directory):
                os.makedirs(directory)
            file_path = os.path.join('./3Dfiles/', f"{name}.xyz")
            with open(file_path, "w") as file:
                file.write(xyz)
            return xyz
        except IndexError:
            return None

    def sdf_to_zmat(self, sdf, smiles):
        """Converts SDF to zmat"""
        name = self.smiles_to_name2(smiles)
        directory = './3Dfiles/'
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("sdf", "gzmat")
            obConv.ReadString(obMol, sdf)
            zmat = obConv.WriteString(obMol)
            if not os.path.exists(directory):
                os.makedirs(directory)
            file_path = os.path.join('./3Dfiles/', f"{name}.txt")
            with open(file_path, "w") as file:
                file.write(zmat)
            return zmat
        except IndexError:
            return None

The `get_sdf()` function is a function that manages the fetching of SDF files in the `3Dfiles` folder. The function uses two arguments : `identifier` and `identifier_type`. <br>
`Identifier_type` is a selection of different chemical identifiers such as the name, SMILES, InChI, InCh, InChIKey or CID. <br>
`Identifier` is the name/notation of the molecule in the any identifier type. <br>

For example let's take benzene. <br>
We could write it in different manners: <br>

>`identifier = "C1=CC=CC=C1"` <br>
>`identifier_type = "SMILES"`

or

>`identifier = "241"` <br>
>`identifier_type = "CID"`

In [73]:
def get_sdf(identifier, identifier_type):
    directory = './3Dfiles/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Define the file path for the SDF file
    sdf_file = os.path.join(directory, f"{identifier}.sdf")

    # Check if the SDF file already exists
    if not os.path.exists(sdf_file):
        # If not, convert SMILES to SDF and save it
        molecule = converter(identifier, identifier_type, cache)
        sdf = molecule.convert('sdf')
        if not sdf:
            print("Failed to convert molecule to SDF.")
            return None
        
        with open(sdf_file, 'w') as f:
            f.write(sdf)
    else:
        # If the SDF file exists, read it
        with open(sdf_file, 'r') as f:
            sdf = f.read()

    return sdf

In [74]:
def get_smiles_from_name(common_name):
    try:
        compounds = pcp.get_compounds(common_name, 'name')
        return compounds[0].isomeric_smiles
    except IndexError:
        return None

def is_valid_molecule(molecule_name):
    smiles = get_smiles_from_name(molecule_name) if not Chem.MolFromSmiles(molecule_name) else molecule_name
    molecule = Chem.MolFromSmiles(smiles)
    return molecule is not None

These are parameters for the 3D viewer *inside* the notebook, not for streamlit.

In [75]:
style_dropdown = Dropdown(
    options=['line', 'stick', 'sphere','All'],
    value='All',
    description='Style:')

linewidth_slider = FloatSlider(
    value=2,
    min=1,
    max=10,
    step=1,
    continuous_update=False,
    description='Line Width')

radius_slider = FloatSlider(
    value=0.2,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='Atomic radius size') 

scale_slider = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='Sphere size')

Similarly, these are parameters for the 3D viewer *inside* the notebook, not for streamlit.

In [76]:
def update_visibility(style):
    if style == 'line':
        linewidth_slider.layout.visibility = 'visible'
        radius_slider.layout.visibility = 'hidden'
        scale_slider.layout.visibility = 'hidden'
    elif style =='stick':
        linewidth_slider.layout.visibility = 'hidden'
        radius_slider.layout.visibility = 'visible'
        scale_slider.layout.visibility = 'hidden'
    elif style =='sphere':
        linewidth_slider.layout.visibility = 'hidden'
        radius_slider.layout.visibility = 'hidden'
        scale_slider.layout.visibility = 'visible'
    elif style == 'All':
        linewidth_slider.layout.visibility = 'visible'
        radius_slider.layout.visibility = 'visible'
        scale_slider.layout.visibility = 'visible'

This function takes care of creating the model for the `view3d` function.

In [77]:
def add_model(view, sdf, style, linewidth, radius, scale):
    """Add the SDF model to the 3Dmol view and apply the given style."""
    try:
        view.addModel(sdf, 'sdf')
    except Exception as e:
        raise RuntimeError(f"Error adding model to the view: {e}")

    view.setBackgroundColor('#000000')
    if style == 'line':
        view.setStyle({'line': {'linewidth': linewidth}})
    elif style == 'stick':
        view.setStyle({'stick': {'radius': radius}})
    elif style == 'sphere':
        view.setStyle({'sphere': {'scale': scale}})
    elif style == 'All':
        view.setStyle({'line': {'linewidth': linewidth}}, viewer=(0,0))
        view.setStyle({'stick': {'radius': radius}}, viewer=(0,1))
        view.setStyle({'sphere': {'scale': scale}}, viewer=(0,2))

The `view3D()` is a 3D visualization tool that renders molecules and allows the user to interact with them, such as zooming or changing the viewing style. This function is separate from the `add_model` function in order to reduce computation time when changing parameters, as to not constantly load the model when changing them.

It takes as input : `data_cache` which is the caching function, `identifier` and `identifier_type` which are the same as described earlier. The remaining parameters are for the interface. `radius` is to change the radius of the sphere for the 'stick' representations. `linewidth` *should* be changing the width of the 'line' representations, however this functionality doesn't seem to work and we did not find a way around it. `scale` changes the radius of the spheres in the 'sphere' representation.
<br>
<br>
To test the function, write what the name or idientifier of the molecule you want next to `identifier` (here we took ethanol as an example). An interface will open below where you can directly change the name of the molecule without changing the code. You can also freely change how the molecule renders using three different visualization types: 'line', 'stick' and 'sphere'. You can also rotate the molecule if you click and hold on the molecule and move your mouse. You can also zoom by using the scrolling wheel or using two fingers on your trackpad.

In [78]:
def view3D(data_cache, identifier, identifier_type, style='All', linewidth='1', radius='0.2', scale='1'):
    """Visualize a molecule in 3D"""

    if identifier not in sdf_cache:  # Check if the SDF file is already cached
        sdf = get_sdf(identifier, identifier_type)  # Fetch the SDF file for the new molecule identifier
        if not sdf:
            print("Failed to fetch SDF file.")
            return None
        sdf_cache[identifier] = sdf # Cache the loaded SDF file
    else:
        sdf = sdf_cache[identifier]
    
    view = py3Dmol.view(width=1000, height=1000)    # 3D visualization
    add_model(view, sdf, style, linewidth, radius, scale) # Parameters
    view.zoomTo()   # Parameter so the user can zoom/dezoom on the molecule
    
    if style == 'All':
        view = py3Dmol.view(width=1500, height=800, viewergrid=(1,3), linked=True)
        add_model(view, sdf, 'All', linewidth, radius, scale)
    return view

interact(view3D, data_cache=fixed(cache), #Example with ethanol
         identifier='ethanol',
         identifier_type=['Name', 'SMILES', 'InChi', 'InChiKey', 'CID'],
         style=style_dropdown,
         linewidth=linewidth_slider,
         radius=radius_slider,
         scale=scale_slider)

style_dropdown.observe(lambda change: update_visibility(change['new']), names='value')

interactive(children=(Text(value='ethanol', description='identifier'), Dropdown(description='identifier_type',…

`get_pdb()` is the analogous function to `get_sdf`, but for proteins. It fetches a pdb file in the `PDBfiles` folder using only 1 argument which is the 4 character pdb code.

In [79]:
def get_pdb(identifier):
    directory = './PDBfiles/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Define the file path for the PDB file
    pdb_file = os.path.join(directory, f"{identifier}.pdb")

    # Check if the PDB file already exists
    if not os.path.exists(pdb_file):
        # Fetch the PDB file from the RCSB PDB database
        pdbl = PDBList()
        pdbl.retrieve_pdb_file(identifier, pdir=directory, file_format='pdb')
        print(f"PDB file '{pdb_file}' not found.")
        print(f"Current directory contents: {os.listdir(directory)}")
        
        # Rename the file to match the identifier
        fetched_file = os.path.join(directory, f"pdb{identifier.lower()}.ent")
        if os.path.exists(fetched_file):
            os.rename(fetched_file, pdb_file)
    
    try:
        with open(pdb_file, 'r') as f:
            pdb_content = f.read()
    except Exception as e:
        print(f"Error reading PDB file '{pdb_file}': {e}")
        return None
    
    return pdb_content

These are parameters for the 3D viewer *inside* the notebook, not for streamlit.

In [80]:
def update_visibility_protein(style):
    if style == 'cartoon':
        radius_protein.layout.visibility = 'hidden'
        scale_protein.layout.visibility = 'hidden'
    elif style =='stick':
        radius_protein.layout.visibility = 'visible'
        scale_protein.layout.visibility = 'hidden'
    elif style =='sphere':
        radius_protein.layout.visibility = 'hidden'
        scale_protein.layout.visibility = 'visible'

In [81]:
identifier_widget = Text(
    value='1ZNI',  # Default value
    placeholder='Enter identifier...',
    description='Identifier:',
    disabled=False
)

style_protein = Dropdown(
    options=['cartoon', 'stick', 'sphere',],
    value='cartoon',
    description='Style:')

color_protein = Dropdown(
    options=['spectrum', 'chain', 'element'],
    value='spectrum',
    description='Color:'
)

radius_protein = FloatSlider(
    value=0.2,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='Atomic radius size') 

scale_protein = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='Sphere size')

asa_checkbox = Checkbox(
    value = False,
    description = 'Add surface'
)

asa_slider = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    continuous_update=False,
    description='ASA surface opacity')

This function takes care of creating the model for the `viewProtein` function.

In [82]:
def add_protein_model(view, pdb_content, global_style, global_color, global_radius, global_scale, surface, opacity):
    """Add the PDB model to the 3Dmol view and apply the given style."""
    try:
        view.addModel(pdb_content, 'pdb')
    except Exception as e:
        raise RuntimeError(f"Error adding model to the view: {e}")
    view.setBackgroundColor('#000000')
    
    # Apply global styles
    if global_style == 'cartoon':
        view.setStyle({'cartoon': {'color': global_color}})
    elif global_style == 'stick':
        view.setStyle({'stick': {'colorscheme': global_color, 'radius': global_radius}})
    elif global_style == 'sphere':
        view.setStyle({'sphere': {'colorscheme': global_color, 'scale': global_scale}})
    
    if surface:
        view.addSurface(py3Dmol.VDW, {'opacity': opacity, 'color': 'spectrum'})

An analogous function to `view3D`, the `viewProtein` visualizer lets the user interact with a 3D model of a chosen protein. `cartoon` lets the user view the classic ribbon style view of a protein, `stick` lets the user see the atoms in a stick view and `sphere` as spheres. The color used to view the protein can also be changed, either choosing `spectrum`, `chain` to separate the protein into its composite chains or `element` to colour the atoms as their elements.

An active surface area checkbox was added (`Add surface`) to view it, the opacity can also be changed. 

Here again the `add_protein_model` function was separated from `viewProtein` to improve computation time.

In [83]:
def viewProtein(identifier, global_style='cartoon', global_color='spectrum', global_radius=0.2, global_scale=1, surface=False, opacity=1):
    """Visualize a molecule in 3D"""
    
    if identifier not in protein_cache:
        # Fetch the PDB file for the new protein identifier
        pdb_content = get_pdb(identifier)
        if not pdb_content:
            print("Failed to fetch PDB file.")
            return None
        # Cache the loaded PDB file
        protein_cache[identifier] = pdb_content
    else:
        pdb_content = protein_cache[identifier]
    
    view = py3Dmol.view(width=800, height=800)
    add_protein_model(view, pdb_content, global_style, global_color, global_radius, global_scale, surface, opacity)
    view.zoomTo()
    return view

interact(viewProtein,
         identifier=identifier_widget,
         global_style=style_protein,
         global_color=color_protein,
         global_radius=radius_protein,
         global_scale=scale_protein,
         surface=asa_checkbox,
         opacity=asa_slider)

style_protein.observe(lambda change: update_visibility_protein(change['new']), names='value')
#update_chain_select('1ZNI')

interactive(children=(Text(value='1ZNI', description='Identifier:', placeholder='Enter identifier...'), Dropdo…

In [84]:
indexs_checkbox = Checkbox(
    value = False,
    description = 'Index atoms (small)'
)

indexl_checkbox = Checkbox(
    value = False,
    description = 'Index atoms (large)'
)

stereo_checkbox = Checkbox(
    value = False,
    description = 'Show stereochemistry' 
)

substructure_input = Text(
    value='',
    description='Enter SMILES substructure'
)

abbr_checkbox = Checkbox(
    value = False,
    description = 'Use abbreviations'
)

These functions will be used for the `view2D`function.
<br>

`index_atoms` shows the index of the atoms
<br>
`indexl_atoms` shows the labels of the atoms
<br>
`stereo_atoms` *should* show the stereochemistry of the atoms but it does not work
<br>
`get_substructure` to search if there's a specific substructure inside the molecule
<br>
`vis_abbr` to show abbreviations instead of drawing the molecule (for example replaces -CH3 by -Me)

In [85]:
def indexs_atoms(molecule):
    IPythonConsole.drawOptions.addAtomIndices = True
    return molecule

def indexl_atoms(molecule):
    for atom in molecule.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return molecule

def stereo_atoms(molecule):
    IPythonConsole.drawOptions.addStereoAnnotation = True
    rdCIPLabeler.AssignCIPLabels(molecule)
    return molecule

def get_substructure(molecule, smarts):
    substructure = Chem.MolFromSmarts(smarts)
    return molecule.GetSubstructMatches(substructure)

def vis_abbr(molecule):
    abbrevs = rdAbbreviations.GetDefaultAbbreviations()
    return rdAbbreviations.CondenseMolAbbreviations(molecule, abbrevs)

The `view2D` function gives a simpler but still interactive 2D representation of a molecule. The SMILES of the molecule has to be input as it is based on the rdkit MolFromSmiles function. The index of the molecule can be viewed in small numbers or bigger (the larger option does not show which molecule is 0 by default). If a molecule has a stereochemistry it will appear when `Show stereochemistry` is selected. For larger molecules abbreviations can also be toggled for a simpler view.

This function also includes substructure matching, just input a SMILES string and the substructure will be highlighted on the molecule! An error can appear when inputing `=` which is normal, as it updates in real time

In [86]:
# Function to add indices to atoms
def indexs_atoms(molecule):
    for atom in molecule.GetAtoms():
        atom.SetProp('atomLabel', str(atom.GetIdx()))
    return molecule

# Function to add labels to atoms
def indexl_atoms(molecule):
    for atom in molecule.GetAtoms():
        atom.SetProp('atomLabel', atom.GetSymbol() + str(atom.GetIdx()))
    return molecule

# Function to handle stereochemistry annotations
def stereo_atoms(molecule):
    IPythonConsole.drawOptions.addStereoAnnotation = True
    rdCIPLabeler.AssignCIPLabels(molecule)
    return molecule

# Function to visualize abbreviations
def vis_abbr(molecule):
    abbrevs = rdAbbreviations.GetDefaultAbbreviations()
    return rdAbbreviations.CondenseMolAbbreviations(molecule, abbrevs)

# Function to get substructure matches
def get_substructure(molecule, substructure):
    substructure_mol = Chem.MolFromSmarts(substructure)
    if substructure_mol is not None:
        return molecule.GetSubstructMatches(substructure_mol)
    else:
        print(f"The substructure '{substructure}' is not valid.")
        return []

def view2D(smi, indexs=True, indexl=True, stereo=False, abbreviations=True, substructure=''):
    molecule = Chem.MolFromSmiles(smi)
    if molecule:
        Draw.PrepareMolForDrawing(molecule)  
        if indexs:
            molecule = indexs_atoms(molecule)
        if indexl:
            molecule = indexl_atoms(molecule)
        if stereo:
            molecule = stereo_atoms(molecule)
        elif stereo == False:
            IPythonConsole.drawOptions.addStereoAnnotation = False
        if abbreviations:
            molecule = vis_abbr(molecule)
        if substructure:
            matches = get_substructure(molecule, substructure)
        else:
            molecule = Draw.MolToImage(molecule)  
        display(molecule)  
    else:
        print(f"The SMILES string '{smi}' is not valid.")
        return None

# Function to handle the interaction with checkboxes
def handle_checkbox(smi, indexs, indexl, stereo, abbreviations, substructure):
    return view2D(smi, indexs, indexl, stereo, abbreviations, substructure)

# Widgets for user interaction
indexs_checkbox = Checkbox(description='Show Atom Indices')
indexl_checkbox = Checkbox(description='Show Label Indices')
stereo_checkbox = Checkbox(description='Show Stereochemistry')
abbr_checkbox = Checkbox(description='Show Abbreviations')
substructure_input = Text(description='Substructure')

# Interactive widget
interact(handle_checkbox, smi=Text(value = 'CSCC[C@@H](C(=O)O)N', description='SMILES String'), 
         indexs=indexs_checkbox, 
         indexl=indexl_checkbox, 
         stereo=stereo_checkbox,
         abbreviations=abbr_checkbox,
         substructure=substructure_input)


interactive(children=(Text(value='CSCC[C@@H](C(=O)O)N', description='SMILES String'), Checkbox(value=False, de…

<function __main__.handle_checkbox(smi, indexs, indexl, stereo, abbreviations, substructure)>

Function `identify_chemical_identifier(input_string)` takes as input the identifier name and outputs the converted name of the molecule. It does this by analyzing the string and looking for patterns. For example, in CID there can only be numbers. This work for the following identifier_types : CID, SMILES, InChI, InChIKey and the name.

Let's take for example `input_string = "VNWKTOKETHGBQD-UHFFFAOYSA-N"`, which is the InChIKey for methane
If we parse this string into the function we obtain:

In [53]:
def identify_chemical_identifier(input_string):
    cid_pattern = r'^\d+$'
    smiles_pattern = r'^[CcNnOoPpSsFfClBrIi%0-9=\-\[\]\(\)\/\+\#\$:\.\,\\\/\@\*\@Hh\\\/\|]+$'
    inchi_pattern = r'^InChI=1S?\/[0-9A-Za-z\.\/\-\(\),]+$'
    inchikey_pattern = r'^[A-Z]{14}-[A-Z]{10}-[A-Z]$'
    name_pattern = r'^[a-zA-Z0-9\s\-]+[a-zA-Z0-9\s\-]*$'

    if re.match(cid_pattern, input_string):
        cid = pcp.get_compounds(input_string, 'cid')
        name = cid[0].synonyms
        mol_name = name[0]
        return mol_name
    elif re.match(smiles_pattern, input_string):
        smiles = pcp.get_compounds(input_string, 'smiles')
        name = smiles[0].synonyms
        mol_name = name[0]
        return mol_name
    elif re.match(inchi_pattern, input_string):
        inchi = pcp.get_compounds(input_string, 'inchi')
        name = inchi[0].synonyms
        mol_name = name[0]
        return mol_name
    elif re.match(inchikey_pattern, input_string):
        inchikey = pcp.get_compounds(input_string, 'inchikey')
        name = inchikey[0].synonyms
        mol_name = name[0]
        return mol_name
    elif re.match(name_pattern, input_string):
        if is_valid_molecule(input_string):
            return input_string
        else:
            return "Error"
    else:
        return "Unknown format"

If you got an error message and then the molecule name, this is normal. Because we are caching the files, they may not be yet generated, however you will still get the molecule name.

In [22]:
identify_chemical_identifier('FS(F)(F)(F)(F)F')

'SULFUR HEXAFLUORIDE'

The `pg_from_sdf(identifier, identifier_type)` function takes as input the `identifier` of the molecule as well as its `identifier_type` and outputs the point group of the molecule. <br>

It works by converting the input into a sdf file, and then getting the 3D coordinates of all the molecules. Then, using the `pymsym` package, we can use the `pymsym.get_point_group()` function which takes as input the coordinates and atomic numbers of all molecules. It then do some mathematics calculations to **estimate** the point group. <br>

I am putting an accent on the word "estimate" because this function is not very precise. I had lot of false results. I believe it does not support high symmetry point groups like Oh, Td, Ih etc. For example, if we try to get the point group of methane, we obtain "Kh" which is the point group of a sphere, instead of Td. Another example is SF6, we obtain D6h for it which is wrong and should be in reality Oh.
<br>
<br>
Therefore we coded `get_molecule_code` which will search for the point group of the molecule in an existing data set (json file) which was found on the Internet. <br>

The way we implemented it in the streamlit app is that it first searches in the json file and if it does not found the molecule, it will then call the other function, `pg_from_sdf` to estimate the point group.

In [30]:
def get_molecule_code(molecule_name, file_path='molecules.json'):
    with open(file_path, 'r') as file:
        data = json.load(file)
        for molecule in data["molList"]:
            if molecule["name"].lower() == molecule_name.lower():
                pg = molecule["pg"]
                return f"Point group of {molecule_name} is {pg}"
    return None
    
def pg_from_sdf(identifier, identifier_type):
    sdf_string = get_sdf(identifier, identifier_type)
    mol = Chem.MolFromMolBlock(sdf_string)
    mol_name = identify_chemical_identifier(input_string=identifier)
    if mol is not None:
        conf = mol.GetConformer()
        coordinates_list = []
        atomic_numbers = []
        for atom in mol.GetAtoms():
            aid = atom.GetIdx()
            pos = conf.GetAtomPosition(aid)
            coordinates_list.append([pos.x, pos.y, pos.z])
            atomic_numbers.append(atom.GetAtomicNum())
        
        pg = pymsym.get_point_group(atomic_numbers=atomic_numbers, positions=coordinates_list)
        return f"Point group of {mol_name} is {pg}"
    else:
        return "Invalid SDF input. Please provide a valid input."


For example if we want to find the point group of methane, it will search into the json file and return `td`

In [31]:
pg = get_molecule_code("methane")
print(pg)

Point group of methane is td


Another example if we want to find the point group of SF6 using the json file.

In [32]:
molecule_name = identify_chemical_identifier("17358")
pg = get_molecule_code(molecule_name)
print(pg)

Point group of SULFUR HEXAFLUORIDE is oh


And if the the molecule is not in the json, in the streamlit app it will call the `pg_from_sdf` function. For example, if we take : "Tetracene"

In [41]:
identifier1 = "CCO"
identifier_type1 = "smiles"
po = pg_from_sdf(identifier=identifier1, identifier_type="smiles")
print(po)

Point group of ethanol is Cs
